In [1]:
from datasets import SyntheticDataset
from crowd_evaluation import ConfidenceEvaluatorC, VoteAggregator, OldEvaluator
import numpy as np

In [2]:
num_workers = 10

for i in range(10):
    print(stdr(i*10) + '% of workers have ')
    p_true = np.random.choice([0.1, 0.3], p=[i/10, 1-(i/10)],size=num_workers)
    dataset = SyntheticDataset(10000, num_workers, sample_fractions=[1]*num_workers, p_true=p_true)

    evaluator = ConfidenceEvaluatorC(dataset)
    ps, confs = evaluator.evaluate_workers_with_confidence(dataset.workers)

In [4]:
ps, confs

(array([0.09481797, 0.10759041, 0.29407573, 0.10905329, 0.10587706,
        0.29295155, 0.10692994, 0.30016267, 0.1058267 , 0.10051227,
        0.28503004, 0.30233675, 0.09711064, 0.09562733, 0.30275726,
        0.30813056, 0.30440468, 0.29404747, 0.29060644, 0.31258562],
       dtype=float32),
 array([0.01532876, 0.01403933, 0.01575967, 0.01400133, 0.00652542,
        0.0075935 , 0.01302642, 0.00739347, 0.01189923, 0.00658251,
        0.01453413, 0.01420352, 0.01432268, 0.0061214 , 0.01451692,
        0.01405237, 0.0147819 , 0.01589864, 0.00723715, 0.01511974],
       dtype=float32))

In [5]:
np.abs(ps - dataset.get_true_error_rates_for_workers(dataset.workers))

array([0.00518203, 0.00759041, 0.00592427, 0.00905329, 0.00587706,
       0.00704845, 0.00692994, 0.00016267, 0.0058267 , 0.00051227,
       0.01496996, 0.00233675, 0.00288936, 0.00437267, 0.00275726,
       0.00813056, 0.00440468, 0.00595253, 0.00939356, 0.01258562])

In [6]:
blocklist = np.argwhere((ps+confs)>0.15)
blocklist

array([[ 2],
       [ 5],
       [ 7],
       [10],
       [11],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19]])

In [7]:
aggregator = VoteAggregator(dataset)
majority_vote = aggregator.majority_vote(dataset.samples)
smart_vote = aggregator.weighted_vote(dataset.samples, dataset, ps, 0.5)
smart_vote_blocked = aggregator.weighted_vote(dataset.samples, dataset, ps, 0.5, blocked_workers=blocklist)
truth = dataset.get_ground_truth_for_samples(dataset.samples)

In [8]:
np.count_nonzero(np.where(truth == majority_vote, 1, 0)) / len(truth)

0.9992

In [9]:
np.count_nonzero(np.where(truth == smart_vote, 1, 0)) / len(truth)

0.9998

In [10]:
np.count_nonzero(np.where(truth == smart_vote_blocked, 1, 0)) / len(truth)

0.9992